In [2]:
import sys
sys.path.append('..')

In [3]:
import random
from copy import deepcopy
from library.solution import Solution

## Genetic Algorithms

Genetic Algorithms (GAs) are a class of optimization algorithms inspired by **natural selection** and **evolutionary principles**. They are used to find near-optimal solutions to complex problems, especially when traditional methods struggle due to high-dimensional or non-differentiable search spaces.

GAs operate by evolving a population of candidate solutions over multiple iterations (called generations), using biologically inspired operations:
- **Selection**: Choosing the best individuals based on a fitness function.
- **Crossover (Recombination)**: Combining two parent solutions to create new offspring.
- **Mutation**: Introducing small random changes to maintain diversity.

### Pseudo-code

1. Initialize a population P of **N** individuals/solutions (usually at random)
2. Repeat until termination condition (**max number of generations**):
   1. Create an empty population P'
   2. If using elitism, insert the best individual from P into P'
   3. Repeat until P' contains N individuals:
      1. Choose 2 individuals from population P using a **selection algorithm**
      2. Choose an operator between crossover and replication with probabilities **$P_c$** and $1-P_c$, respectively
      3. Apply the operator to the individuals to generate the offspring
      4. Apply mutation to the offspring. The mutation operator has an hyperparameter **$P_m$** (we'll see what this means for different mutation operators later)
      5. Insert the mutated individuals into P'
   4. Replace P with P'
3. Return the best individual in P


### Algorithm Implementation

Let's implement the genetic algorithm function. These are the arguments this function will receive:
- `initial_population`: List of individuals (randomly generated solutions)
- `max_gen`: Maximum number of generations
- `selection_algorithm`: A function that receives a population, selects one individual based on fitness and returns it
- `maximization`: Boolean that indicates if we're solving a maximization or minimization problem
- `xo_prob`: Probability of crossover (usually big)
- `mut_prob`: Probability of mutation (usually small)
- `elistism`: A boolean that indicates if elitism should be used or not

In [ ]:
from typing import Callable

def get_best_ind(population: list[Solution], maximization: bool):
    fitness_list = [ind.fitness() for ind in population]
    if maximization:
        return population[fitness_list.index(max(fitness_list))]
    else:
        return population[fitness_list.index(min(fitness_list))]

def genetic_algorithm(
    initial_population: list[Solution],
    max_gen: int,
    selection_algorithm: Callable,
    maximization: bool = False,
    xo_prob: float = 0.9,
    mut_prob: float = 0.1,
    elitism: bool = True,
    verbose: bool = False,
):
    # 1. Initialize a population with N individuals
    population = initial_population

    # 2. Repeat until termination condition
    for gen in range(1, max_gen + 1):
        if verbose:
            print(f'-------------- Generation: {gen} --------------')

        # 2.1. Create an empty population P'
        new_population = []

        # 2.2. If using elitism, insert best individual from P into P'
        if elitism:
            new_population.append(deepcopy(get_best_ind(initial_population, maximization)))
        
        # 2.3. Repeat until P' contains N individuals
        while len(new_population) < len(population):
            # 2.3.1. Choose 2 individuals from P using a selection algorithm
            first_ind = selection_algorithm(population, maximization)
            second_ind = selection_algorithm(population, maximization)

            if verbose:
                print(f'Selected individuals:\n{first_ind}\n{second_ind}')

            # 2.3.2. Choose an operator between crossover and replication
            # 2.3.3. Apply the operator to generate the offspring
            if random.random() < xo_prob:
                offspring1, offspring2 = first_ind.crossover(second_ind)
                if verbose:
                    print(f'Applied crossover')
            else:
                offspring1, offspring2 = deepcopy(first_ind), deepcopy(second_ind)
                if verbose:
                    print(f'Applied replication')
            
            if verbose:
                print(f'Offspring:\n{offspring1}\n{offspring2}')
            
            # 2.3.4. Apply mutation to the offspring
            first_new_ind = offspring1.mutation(mut_prob)
            # 2.3.5. Insert the mutated individuals into P'
            new_population.append(first_new_ind)

            if verbose:
                print(f'First mutated individual: {first_new_ind}')
            
            if len(new_population) < len(population):
                second_new_ind = offspring2.mutation(mut_prob)
                new_population.append(second_new_ind)
                if verbose:
                    print(f'Second mutated individual: {first_new_ind}')
        
        # 2.4. Replace P with P'
        population = new_population

        if verbose:
            print(f'Final best individual in generation: {get_best_ind(population, maximization)}')

    # 3. Return the best individual in P
    return get_best_ind(population, maximization)


**NOTE:** There are many variations of genetic algorithms. The implementation used in our practical classes and the library follows some choices. For example, before inserting the second mutated individual into P', we check whether it would exceed the population size. This can happen with odd-sized populations since we always insert two individuals at a time. An alternative approach would be to insert the individual regardless and, if the population exceeds the limit, remove the worst-performing individual at the end.

There are also other assumptions for our implementation of the algorithm to run.
- individuals have `fitness`, `crossover` and `mutation` methods
- `crossover` always returns two offspring
- both `crossover` and `mutation` methods return new individuals instead of modifying individuals in-place

### Selection algorithms

Selection is the first main step of a genetic algorithm. Selection algorithms have the following properties:
- are probabilistic
- for any pair of individuals A and B, if A if better than B, then the probability of selecting A must be bigger than the probability of selecting B
- all individuals must have the chance of being selected, even the worst in the population
- when an individual is selected, it remains in population P and a copy is inserted in P'

In class we'll implement **Fitness Proportionate Selection** (or roulette wheel), but there are other techniques like Ranking or Tournament selection.

#### Fitness Proportionate Selection

Probabilistic selection method used in GAs to choose individuals for reproduction. It mimics a roulette wheel, where individuals with higher fitness have a greater chance of being selected, but lower-fitness individuals still have some probability of selection.

Let $N$ be the number of individuals in population $P$ and $F = {f_1, f_2, ..., f_N}$ be the set of fitness values of the indiiduals in the population. For an individual $i$ in the population, the probability of selecting $i$ is:

$$P(selecting\ i) = \frac{f_i}{\sum_{j=1}^{N} f_j}$$

![Fitness Proportionate Selection Implementation](images/fps.png)

Our implementation fo this selection algorithm will be a function that receives two arguments:
- `population`: A list of individuals / solutions. These must have a `fitness()` method.
- `maximization`: Boolean that indicates if we're solving a maximization or minimization problem

In [ ]:
def fitness_proportionate_selection(population: list[Solution], maximization: bool):
    total_fitness = sum([ind.fitness() for ind in population])

    if maximization:
        fitness_values = [ind.fitness() for ind in population]
    else:
        # Minimization: Use the inverse of the fitness value
        # Lower fitness should have higher probability of being selected
        fitness_values = [1 / ind.fitness() for ind in population]

    total_fitness = sum(fitness_values)
    # Generate random number between 0 and total
    random_nr = random.uniform(0, total_fitness)
    box_boundary = 0
    # For each individual, check if random number is inside the individual's "box"
    for ind_idx, ind in enumerate(population):
        box_boundary += fitness_values[ind_idx]
        if random_nr <= box_boundary:
            return deepcopy(ind)